# PatMatch nucleic handling flags demystified

-----
***TL;DR:***

### **<font color="red">If you want the pattern search to be performed on both strands of the supplied sequence, as is the default of the web-based PatMatch tools, you actually want to use the `-c` flag when authoring the command.</font>**

If that is sufficient for your uses, feel fee to skip this notebook and continue on matching patterns with PatMatch. If you want to know more...

-----
The notebook contents below illustrate clearly, step-by-step why I suggest this.

## The situation

I found the two choices for flags for PatMatch covered in PatMatch [USAGE](PatMatch initial demo and introduction.ipynb#Usage) a little unclear given typically options of web-based PatMatch list three choices. For example the associated option for [PatMatch for the Arabidopsis genome](http://www.arabidopsis.org/cgi-bin/patmatch/nph-patmatch.pl) looks like this:

>If DNA, Strand:
- both strands
- Watson(given)
- Crick(reverse complement)


The listing for that option at the site for [PatMatch with the S. cerevisiae genome](https://www.yeastgenome.org/nph-patmatch) looks like this:

> If DNA, Strand:
- Both strands
- Strand in dataset
- Reverse complement of strand in dataset


Yet [the USAGE](PatMatch initial demo and introduction.ipynb#Usage) for the stand-alone, command line based PatMatch simply lists:

>Run PatMatch with the command:  
perl patmatch.pl [residue] [pattern] [sequence file] [number of mismatches] [mismatch types]
Residue types:  
-n for nucleotide  
-c for complementary strand  
-p for protein  

Showing only two options for nucleic. 

**Three options in the web-based offerings but only two in the command line program?**

Given the situation, I initially I thought maybe one was for one strand and the other for the other strand and I could list both in the command. Or perhaps, I was expected to run first with `-n` and then with `-c` and combine the results. However, a few tests show that neither of those are the case.

-----

## Testing the various flag options

### Preparation

Example data will be obtained and prepared for use.

In [ ]:
!curl -O https://downloads.yeastgenome.org/sequence/S288C_reference/chromosomes/fasta/chr15.fsa
!perl ../patmatch_1.2/unjustify_fasta.pl chr15.fsa

### Testing the basic nucleic command

Now to test some options.

First try with the `-n` flag.

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -n "DDWDWTAWAAGTARTADDDD" chr15.fsa.prepared 

That just gives one sequence:

```
>tpg|BK006948.2|:[541299,541318]
GTTATTAAAAGTAATATTGT 
```

### Testing combining

Trying a couple of ways to add the `-c` flag to this shows that they seem to be exclusive

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -nc "DDWDWTAWAAGTARTADDDD" chr15.fsa.prepared

`!perl ../patmatch_1.2/patmatch.pl -nc "DDWDWTAWAAGTARTADDDD" chr15.fsa.prepared` seems to return nothing. Not even a warning.

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -n -c "DDWDWTAWAAGTARTADDDD" chr15.fsa.prepared

Seems no way to use both `-n` and `-c` flags in same call to `patmatch.pl`.

### Testing with the `-c` flag

Let's go ahead and try the `-c` flag and see what happens.

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chr15.fsa.prepared

That returns two sequences:

```
>tpg|BK006948.2|:[157831,157812]
AAATATAAAAGTAGTAAGAA 
>tpg|BK006948.2|:[541299,541318]
GTTATTAAAAGTAATATTGT 
```

In other words we now see two sequences where we only saw one with the `-n` flag. Indeed, we see the one, `GTTATTAAAAGTAATATTGT [541299,541318]`, we obtained when we used the `-n` flag originally and have gained another. 

And so it looks strongly suggestive that the `-c` flag options means it is **for the complementary strand in addition to the strand in the dataset**. But can we prove that here? Can we see what the result is for just the complementary strand?

As discussed above we seem to be out of options that we can use for nucleic acid with the command line `patmatch` command, but independent of anything involving PatMatch we can actually convert our sample sequence to a reverse complement using the popular [Biopython module](http://biopython.org/wiki/Seq), and then try with that where we again use the `-n` flag.

In [ ]:
## CONVERT TO REVERSE COMPLEMENT USING BIOPYTHON **
from Bio import SeqIO
from Bio.Seq import Seq # for reverse complement

def get_seq_from_file(file_name):
    '''
    takes a file name and gets the sequence RECORD. 
    '''
    fasta_iterator = SeqIO.parse(file_name, "fasta")
    # Use of `next()` on next line to get first FASTA -formatted sequence is 
    # based on http://biopython.org/DIST/docs/api/Bio.SeqIO-module.html
    # I think difference from `SeqIO.read()` in this approach is that it won't
    # give an error of more than one entry is in the html.
    # I found I needed `StringIO()` or got issues with trying to handle long file name.
    record = next(fasta_iterator)
    #return record.seq
    #return record.id
    return record
    
# Read FASTA file
input_file_name = "chr15.fsa"
sequence_record = get_seq_from_file(input_file_name)
#print(sequence_record) #FOR DEBUGGING

# Get reverse complement
seq_rev_compl_record = sequence_record.reverse_complement()
seq_rev_compl_record.id = sequence_record.id #record needs id for writing FASTA
#print(seq_rev_compl_record) #FOR DEBUGGING

# Save FASTA file for reverse complement
output_file_name = "chr15.revcompl.fa"
SeqIO.write(seq_rev_compl_record, output_file_name, "fasta");

Next prepare the reverse complement made in the last cell for use with `patmatch.pl`.

In [ ]:
!perl ../patmatch_1.2/unjustify_fasta.pl chr15.revcompl.fa

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -n "DDWDWTAWAAGTARTADDDD" chr15.revcompl.fa.prepared


That just simply returns one sequence:

```
>tpg|BK006948.2|:[933461,933480]
AAATATAAAAGTAGTAAGAA 
```

This is the same sequence as the additional sequence we saw gained when `!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chr15.fsa.prepared` was run after running with the `-n` flag. Just the numbering appears different because numbering along reverse complement being returned. Chromosome 15 is 1091291 bps long and so can switch back numbering using 1091291 - 933461 + 1 = 157831.

That proves that that `-c` means it is for the complementary strand **in addition to** the strand in the dataset. And perhaps, `-b`, along the lines of `-both`, might have been a better letter choice for the flag designation.

(I didn't see an option in the USAGE infromation in the README to actually do the third option that the web-based PatMatch implementations typically offer, that being searching for the pattern in the reverse complement strand only. The sites must supply the reverse complement data and trigger using that with the `-n` option internally when user's choose just to look at the reverse complement, it seems. And we also have shown here that converting to the reverse complement is easily done within Python as well, and so those options are are available from the command line, too. It's just not as obvious how to that here as with the web-based implementations.)

In summary, this is why I point out in the main notebook that typically you probably want the `-c` flag for nucleic acid pattern matching. Here is that note repeated here:

### **<font color="red">If you want the pattern search to be performed on both strands of the supplied sequence, as is the default of the web-based PatMatch tools, you actually want to use the `-c` flag when authoring the command.</font>**